In [1]:
############################################################################
# 엘라스틱서치 8.x.x 버전일때 테스트 예시.
#
#===========================================================================
# [api_key 생성 방법]
# 출처 : https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/getting-started-python.html
# => kibana 접속 : localhost:5601->메뉴에서 Search>elasticsearch 클릭
# ->화면중간에 API KEY 생성 클릭> 이후 Name 입력하고, Contol security privileges(보안접근권한) 설정후 [Create API Key] 입력
# => python 코드에서는 api_key 인자로 encoded 값을 입력해야 함.
#{
#  "id": "1Rtxxxxxxxxxxxxxxxxx",
#  "name": "apikey",
#  "api_key": "3Pfxxxxxxxxxxxxxx",
#  "encoded": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx===",
#  "beats_logstash_format": "1Rtoxxxxxxxxxxxxxxxxx"
#}

# scikit-extra 설치
# !pip install scikit-learn-extra

# elasticsearch 버전 보기 
# !pip show elasticsearch

# pip로 elasticsearch 8.14.3으로 업데이트 
# !pip install --upgrade elasticsearch
############################################################################
import torch
import time
import os
import numpy as np
import sys

sys.path.append('../')
from utils import MyUtils
from es import My_ElasticSearch

# settings.yaml 설정값 불러옴.
myutils = MyUtils(yam_file_path='../data/es_settings.yaml')
settings = myutils.get_options()

# elastic search 설정
es_url=settings['ES_URL']
es_index_name=settings['ES_INDEX_NAME']
es_index_file_path=settings['ES_INDEX_FILE_PATH']
es_api_key=settings['ES_API_KEY']
myes = My_ElasticSearch(es_url=es_url, index_name=es_index_name, index_file_path=es_index_file_path, api_key=es_api_key, create=True)
print(f'*ES정보: {myes}')

/usr/local/lib/python3.10/dist-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.130:9201' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


*[create_index] self.es.indices.delete
*[create_index] self.es.indices.create
*ES정보: <es.main.My_ElasticSearch object at 0x7fa4227d47c0>


In [2]:
# 인덱스에 데이터 추가
doc1 = {'rfile_name':'제주도', 'rfile_text':'제주도는 대한민국이 남쪽에 있는 섬이다.'}

res=myes.insert(doc=doc1)
print(f'rs:{res}')

*doc:{'rfile_name': '제주도', 'rfile_text': '제주도는 대한민국이 남쪽에 있는 섬이다.'}, <class 'dict'>
rs:{'_index': 'docs_kpf-sbert-128d-10-1-keword', '_id': 'dLE7IZEBUfYHPRM4Vjiy', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# 한꺼번에 인덱싱
es_index_name=settings['ES_INDEX_NAME']
op_data = {"index": {"_index": es_index_name}}
print(f"*op_data:{op_data}")

doc2 = {'rfile_name':'한라산', 'rfile_text':'한라산은 제주도에 있는 휴화산이다.'}
doc3 = {'rfile_name':'서귀포', 'rfile_text':'대한민국 제주도 최남단에 있는 도시이다.'}
doc4 = {'rfile_name':'제주시', 'rfile_text':'제주도의 행정/경제 중심 도시이다.'}
doc5 = {'rfile_name':'성산읍', 'rfile_text':'제주도 동쪽에 있는 읍으로, 성산일출봉과 제2공항 후보지로 유명하다.'}

# 출처 : https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html 참고
docs:list=[]
docs.append(op_data)
docs.append(doc2)
docs.append(op_data)
docs.append(doc3)
docs.append(op_data)
docs.append(doc4)
docs.append(op_data)
docs.append(doc5)

print(f"*docs:{docs}")

res=myes.RAG_bulk(docs=docs)
print(f'rs:{res}')

*op_data:{'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword'}}
*docs:[{'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword'}}, {'rfile_name': '한라산', 'rfile_text': '한라산은 제주도에 있는 휴화산이다.'}, {'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword'}}, {'rfile_name': '서귀포', 'rfile_text': '대한민국 제주도 최남단에 있는 도시이다.'}, {'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword'}}, {'rfile_name': '제주시', 'rfile_text': '제주도의 행정/경제 중심 도시이다.'}, {'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword'}}, {'rfile_name': '성산읍', 'rfile_text': '제주도 동쪽에 있는 읍으로, 성산일출봉과 제2공항 후보지로 유명하다.'}]
rs:{'errors': False, 'took': 7, 'items': [{'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword', '_id': 'dbE7IZEBUfYHPRM4Yjid', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'docs_kpf-sbert-128d-10-1-keword', '_id': 'drE7IZEBUfYHPRM4Yjid', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
#데이터 검색
query="제주도"
min_score=0.1
res=myes.BM25_search_docs(query=query, min_score=min_score)
print(f'rs:{res}')

rs:[{'rfile_name': '제주시', 'score': 0.19570608}, {'rfile_name': '한라산', 'score': 0.18675612}, {'rfile_name': '제주도', 'score': 0.17858894}]


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
# 모든 데이터 불러옴 
script_query = {"match_all": {}}
res=myes.search_docs(script_query=script_query)
print(f'rs:{res}')

rs:[{'rfile_name': '제주도', 'score': 1.0}, {'rfile_name': '한라산', 'score': 1.0}, {'rfile_name': '서귀포', 'score': 1.0}, {'rfile_name': '제주시', 'score': 1.0}, {'rfile_name': '성산읍', 'score': 1.0}]


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
# 인덱스내 데이터 불러옴
data = {'rfile_name': '제주도'}
res=myes.search(data=data)
print(f'rs:{res}')

rs:{'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
# 인덱스에 데이터 업데이트
# =>업데이트할 _id, _type을 알아야함

# 쿼리 검색
data = {'rfile_name': '제주도'}
res=myes.search(data=data)
print(f'rs:{res}')
print('\n')

id_list = []
# 업데이트할 id를 구함
for hits in res['hits']['hits']:
    id=hits['_id']
    #print(f'id:{id}')
    
    id_list.append(id)
    
# 업데이트
for id in id_list:
    
    print(f'id:{id}')
    doc = {'rfile_text': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 70만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    
    res=myes.update(doc=doc, esid=id)
    print(f'res:{res}')
    print('\n')

rs:{'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.3862942, 'hits': [{'_index': 'docs_kpf-sbert-128d-10-1-keword', '_id': 'dLE7IZEBUfYHPRM4Vjiy', '_score': 1.3862942, '_source': {'rfile_name': '제주도', 'rfile_text': '제주도는 대한민국이 남쪽에 있는 섬이다.'}}]}}


id:dLE7IZEBUfYHPRM4Vjiy
res:{'_index': 'docs_kpf-sbert-128d-10-1-keword', '_id': 'dLE7IZEBUfYHPRM4Vjiy', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1}




/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [12]:
# 인덱스내 데이터 삭제
data = {'rfile_name': '제주도'}
res=myes.delete(doc=data)
print(f'rs:{res}')

rs:{'took': 10, 'timed_out': False, 'total': 1, 'deleted': 1, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
# 인덱스 삭제
res=myes.delete_index()
print(f'rs:{res}')

rs:{'acknowledged': True}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
